In [1]:
import numpy as np
import MatrixProductFunctions as mp
from numba import jit
import time
import opt_einsum as oe
import matplotlib.pyplot as plt

class MatrixProductState():
    
    def __init__(self, vector, eps=1e-6, bond_limit=1000):
        if isinstance(vector, np.ndarray):
            self.mps = self.transform_vec_to_mps(vector, eps, bond_limit)
            self.length = np.log2(len(vector))
        else:
            self.mps = vector
            self.length = len(vector)
            
        self.norm = self.__abs__()
        self.bond = self.get_bond()

    def transform_vec_to_mps(self, vector, eps, bond_limit):
        psi_vec = np.asarray(vector)
        n = int(np.log2(len(psi_vec)))
        if 2**n != len(psi_vec):
            raise ValueError("Длина вектора должна быть степенью 2.")
            
        mps = []
        psi_matrix = psi_vec.reshape((2,-1))

        for i in range(n - 1):
            psi_svd = np.linalg.svd(psi_matrix, full_matrices=False)
            bond = min((psi_svd.S / psi_svd.S.sum() > eps).sum(), bond_limit)

            psi_svd_U = psi_svd.U.reshape(int(psi_svd.U.shape[0] / 2), 2, int(psi_svd.U.shape[1]))[:, :, :bond]
            mps.append(psi_svd_U)

            pmatrix = np.diag(psi_svd.S[:bond]) @ psi_svd.Vh[:bond, :]
            psi_matrix = pmatrix.reshape(int(pmatrix.shape[0] * 2), int(pmatrix.shape[1] / 2))

        last_tensor = pmatrix.reshape(int(pmatrix.shape[0]), 2, int(pmatrix.shape[1] / 2))
        mps.append(last_tensor)
        return mps
    
    def turn_to_lc_form(self):
        if not isinstance(self.mps, list):
            raise TypeError("Аргументы должны быть списками")

        mps_copy = self.mps.copy()
        left_canonical_mps = [] # Создаём лист для хранения тензоров
        temporary_element = [mps_copy[0]]
        for i in range(len(mps_copy)-1):
            sh = temporary_element[0].shape
            element = temporary_element[0].reshape(sh[0] * 2, sh[2]) # Переводим тензор в матрицу
            element_q, element_r = np.linalg.qr(element) # Делаем сингулярное разложение матрицы
            tensor = element_q.reshape(sh[0], 2, -1) # Переводим матрицу Vh в тензор
            left_canonical_mps.append(tensor) # Вносим тензор в лист хранения тензоров   
            newpmatrx = element_r
            pmatrix = np.tensordot(newpmatrx, mps_copy[i+1], axes = 1) # Перемножаем более левый тензор с перемноженными матрицами U и S
            temporary_element = [pmatrix]
        left_canonical_mps.append(pmatrix) # Добавляем последний тензор в лист хранения тензоров
        return left_canonical_mps
    
    def __abs__(self):
        left_canonical_mps = self.turn_to_lc_form()
        return np.sqrt(np.trace(np.conj(left_canonical_mps[-1])[:, 0, :].T @ left_canonical_mps[-1][:, 0, :]) + np.trace(np.conj(left_canonical_mps[-1])[:, 1, :].T @ left_canonical_mps[-1][:, 1, :]))

    def get_bond(self):
        dim1 = []
        dim2 = []
        for i in self.mps:
            dim1.append(i.shape[0])
            dim2.append(i.shape[-1])
        return max(max(dim1), max(dim2))
    
    def __add__(self, other):

        if not isinstance(other, MatrixProductState):
            raise TypeError("Операнд должен быть MatrixProductState")
        if len(self.mps) != len(other.mps):
            raise ValueError("Длины двух MPS должны быть одинаковы")

        sum_mps = []
        sum_mps.append(np.concatenate([self.mps[0], other.mps[0]], axis = 2).reshape(1, 2, -1))
        for i in range(1, len(self.mps)-1):
            zero_arr1 = np.zeros((other.mps[i].shape[0], 2, self.mps[i].shape[2]))
            zero_arr2 = np.zeros((self.mps[i].shape[0], 2, other.mps[i].shape[2]))
            ff1 = np.concatenate([self.mps[i], zero_arr1], axis=0).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, self.mps[i].shape[2])
            ff2 = np.concatenate([zero_arr2, other.mps[i]], axis=0).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, other.mps[i].shape[2])
            new_element = np.concatenate([ff1, ff2], axis = 2).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, other.mps[i].shape[2] + self.mps[i].shape[2])
            sum_mps.append(new_element)
        sum_mps.append(np.concatenate([self.mps[-1], other.mps[-1]], axis = 0).reshape(self.mps[-1].shape[0] + other.mps[-1].shape[0], 2, 1))
        return MatrixProductState(sum_mps)
    
    def __sub__(self, other):

        if not isinstance(other, MatrixProductState):
            raise TypeError("Операнд должен быть MatrixProductState")
        if len(self.mps) != len(other.mps):
            raise ValueError("Длины двух MPS должны быть одинаковы")

        sub_mps = []
        sub_mps.append(np.concatenate([self.mps[0], -other.mps[0]], axis = 2).reshape(1, 2, -1))
        for i in range(1, len(self.mps)-1):
            zero_arr1 = np.zeros((other.mps[i].shape[0], 2, self.mps[i].shape[2]))
            zero_arr2 = np.zeros((self.mps[i].shape[0], 2, other.mps[i].shape[2]))
            ff1 = np.concatenate([self.mps[i], zero_arr1], axis=0).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, self.mps[i].shape[2])
            ff2 = np.concatenate([zero_arr2, other.mps[i]], axis=0).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, other.mps[i].shape[2])
            new_element = np.concatenate([ff1, ff2], axis = 2).reshape(other.mps[i].shape[0] + self.mps[i].shape[0], 2, other.mps[i].shape[2] + self.mps[i].shape[2])
            sub_mps.append(new_element)
        sub_mps.append(np.concatenate([self.mps[-1], other.mps[-1]], axis = 0).reshape(self.mps[-1].shape[0] + other.mps[-1].shape[0], 2, 1))
        return MatrixProductState(sub_mps)
    
    def __mul__(self, number):
        new_mps = self.mps.copy()
        new_mps[0] = new_mps[0]*number
        return MatrixProductState(new_mps)
    
    def __truediv__(self, number):
        new_mps = self.mps.copy()
        new_mps[0] = new_mps[0]/number
        return MatrixProductState(new_mps)
    
    def __matmul__(self, other):
        
        if self.length != other.length:
            raise ValueError("Длины двух mps должны быть одинаковы")
        
        if isinstance(other, MatrixProductState):   
            mps1 = self.mps
            mps2 = other.mps
            sh1 = mps1[0].shape
            sh2 = mps2[0].shape
            result = np.einsum('ijk,mjl->imkl', np.conj(mps1[0]), mps2[0]).reshape(sh1[0] * sh2[0], sh1[2] * sh2[2])
            for i in range(1, len(mps1)):
                sh1 = mps1[i].shape
                sh2 = mps2[i].shape
                result = result @ np.einsum('ijk,mjl->imkl', np.conj(mps1[i]), mps2[i]).reshape(sh1[0]*sh2[0], sh1[2]*sh2[2])
            result = result.reshape(-1)[0]
            return result
        
        elif isinstance(other, MatrixProductOperator):   
            new_mps = []
            for i in range(int(self.length)):
                new_mps.append(np.einsum("ijnl, mjo -> imnlo", other.mpo[i], self.mps[i]).reshape(other.mpo[i].shape[0] * self.mps[i].shape[0], 2, other.mpo[i].shape[-1] * self.mps[i].shape[-1]))
            return MatrixProductState(new_mps)
        
        else:
            raise TypeError("Неправильное использование аргументов")
    
    def __str__(self):
        return (f"MPS(n_tensors={self.length}, "
                f"max_bond={self.bond})")
    
    def __len__(self):
        return len(self.mps)
    
    def __neg__(self):
        neg_mps = self.mps.copy()
        neg_mps[0] = -neg_mps[0]
        return MatrixProductState(neg_mps)
    
vector_1 = np.random.randn(2**4)
vector_2 = np.random.randn(2**4)
a = MatrixProductState(vector_1, 1e-10)
b = MatrixProductState(vector_2, 1e-10)
c = MatrixProductState(mp.x_mps(5))
print(a)
print(b)
print(c)
#print(a.mps)

MPS(n_tensors=4.0, max_bond=4)
MPS(n_tensors=4.0, max_bond=4)
MPS(n_tensors=5, max_bond=2)


In [2]:
class MatrixProductOperator():
    def __init__(self, list_data):
        if not isinstance(list_data, list):
            raise TypeError("Некорректная инициализация MPO")
        else:
            self.mpo = list_data
            
        self.length = len(list_data)
        self.bond = self.get_bond()
        
    def get_bond(self):
        dim1 = []
        dim2 = []
        for i in self.mpo:
            dim1.append(i.shape[0])
            dim2.append(i.shape[-1])
        return max(max(dim1), max(dim2))
    
    def __matmul__(self, other):
        if self.length != other.length:
            raise ValueError("Длины объектов должны быть одинаковыми")
        
        if isinstance(other, MatrixProductOperator):
            mult_mpo = []
            for i in range(self.length):
                sh1 = self.mpo[i].shape
                sh2 = other.mpo[i].shape
                mult_mpo.append(np.einsum('ijkl, mkop -> imjolp', self.mpo[i], other.mpo[i]).reshape(sh1[0]*sh2[0], 2, 2, sh1[3]*sh2[3]))
            return MatrixProductOperator(mult_mpo)
        
        elif isinstance(other, MatrixProductState):
            new_mps = []
            for i in range(self.length):
                new_mps.append(np.einsum("ijnl, mno -> imjlo", self.mpo[i], other.mps[i]).reshape(self.mpo[i].shape[0] * other.mps[i].shape[0], 2, self.mpo[i].shape[-1] * other.mps[i].shape[-1]))
            return MatrixProductState(new_mps)
        
        else:
            raise TypeError("Неправильное использование аргументов")
            
    def __add__(self, other):
        common_tensor = []
        common_tensor.append(np.concatenate([self.mpo[0], other.mpo[0]], axis = 3).reshape(1, 2, 2, -1))
        for i in range(1, self.length-1):
            zero_arr1 = np.zeros((other.mpo[i].shape[0], 2, 2, self.mpo[i].shape[3]))
            zero_arr2 = np.zeros((self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3]))
            ff1 = np.concatenate([self.mpo[i], zero_arr1], axis=0).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, self.mpo[i].shape[3])
            ff2 = np.concatenate([zero_arr2, other.mpo[i]], axis=0).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3])
            new_element = np.concatenate([ff1, ff2], axis = 3).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3] + self.mpo[i].shape[3])
            common_tensor.append(new_element)
        common_tensor.append(np.concatenate([self.mpo[-1], other.mpo[-1]], axis = 0).reshape(self.mpo[-1].shape[0] + other.mpo[-1].shape[0], 2, 2, 1))
        return MatrixProductOperator(common_tensor)
    
    def __sub__(self, other):
        common_tensor = []
        common_tensor.append(np.concatenate([self.mpo[0], -other.mpo[0]], axis = 3).reshape(1, 2, 2, -1))
        for i in range(1, self.length-1):
            zero_arr1 = np.zeros((other.mpo[i].shape[0], 2, 2, self.mpo[i].shape[3]))
            zero_arr2 = np.zeros((self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3]))
            ff1 = np.concatenate([self.mpo[i], zero_arr1], axis=0).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, self.mpo[i].shape[3])
            ff2 = np.concatenate([zero_arr2, other.mpo[i]], axis=0).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3])
            new_element = np.concatenate([ff1, ff2], axis = 3).reshape(other.mpo[i].shape[0] + self.mpo[i].shape[0], 2, 2, other.mpo[i].shape[3] + self.mpo[i].shape[3])
            common_tensor.append(new_element)
        common_tensor.append(np.concatenate([self.mpo[-1], other.mpo[-1]], axis = 0).reshape(self.mpo[-1].shape[0] + other.mpo[-1].shape[0], 2, 2, 1))
        return MatrixProductOperator(common_tensor)
    
    def __mul__(self, number):
        new_mpo = self.mpo.copy()
        new_mpo[0] = new_mpo[0]*number
        return MatrixProductState(new_mpo)
    
    def __truediv__(self, number):
        new_mpo = self.mpo.copy()
        new_mpo[0] = new_mpo[0]/number
        return MatrixProductOperator(new_mpo)

In [3]:
def compute_mps_layer_from_left_to_right(layer, tensors, optimize):
    new_layer = np.einsum('ab, aij -> jib', layer, tensors[0], optimize=optimize)
    new_layer = np.einsum('abc, cbj -> aj', new_layer, tensors[1], optimize=optimize)
    return new_layer

def compute_mps_layer_from_right_to_left(layer, tensors, optimize):
    new_layer = np.einsum('ab, ija -> ijb', layer, tensors[0], optimize=optimize)
    new_layer = np.einsum('abc, ibc -> ai', new_layer, tensors[1], optimize=optimize)
    return new_layer

def compute_mpo_layer_from_left_to_right(layer, tensors, optimize):
    new_layer = np.einsum('abc, ajk -> kjbc', layer, tensors[0], optimize=optimize)
    new_layer = np.einsum('abcd, cbij -> ajid', new_layer, tensors[1], optimize=optimize)
    new_layer = np.einsum('abcd, dci -> abi', new_layer, tensors[2], optimize=optimize)
    return new_layer

def compute_mpo_layer_from_right_to_left(layer, tensors, optimize):
    new_layer = np.einsum('abc, ija -> ijbc', layer, tensors[0], optimize=optimize)
    new_layer = np.einsum('abcd, ibjc -> aijd', new_layer, tensors[1], optimize=optimize)
    new_layer = np.einsum('abcd, icd -> abi', new_layer, tensors[2], optimize=optimize)
    return new_layer

def compute_reduced_A(left_layers, right_layers, mpo_1, mpo_2, optimize):
    tensor = np.einsum("abc, ijk, bopw, wefj -> aoeicpfk", left_layers, right_layers, mpo_1, mpo_2, optimize=optimize)
    shape = tensor.shape
    return tensor.reshape(shape[0]*shape[1]*shape[2]*shape[3], shape[4]*shape[5]*shape[6]*shape[7])

def compute_reduced_b(left_layers, right_layers, mps_1, mps_2, optimize):
    tensor = np.einsum("ab, jk, apq ,qdj -> bpdk", left_layers, right_layers, mps_1, mps_2, optimize=optimize)
    return tensor.reshape(-1)

def compute_pair_of_tensors(A, b, shape, direction, eps, bond_limit):
    solution = np.linalg.solve(A, b)

    if direction == "left_to_right":
        solution = solution.reshape(shape*2, -1)
    else:
        solution = solution.reshape(-1, shape*2)

    svd_solution = np.linalg.svd(solution, full_matrices = False)
    bond = min((svd_solution.S / svd_solution.S.sum() > eps ).sum(), bond_limit)

    if direction == "left_to_right":
        tensor_1 = svd_solution.U.reshape(shape, 2, -1)[:, :, :bond]
        tensor_2 = (np.diag(svd_solution.S[:bond]) @ svd_solution.Vh[:bond, :]).reshape(tensor_1.shape[-1], 2, -1)
    else:
        tensor_2 = svd_solution.Vh.reshape(-1, 2, shape)[:bond, :, :]
        tensor_1 = (svd_solution.U[:, :bond] @ np.diag(svd_solution.S[:bond])).reshape(-1, 2, tensor_2.shape[0])[:, :, :bond]
    return [tensor_1, tensor_2]

def sweep(mps_train, b_mps, mpo_train, right_mps_layers, right_mpo_layers, eps, bond_limit, optimize):

    n = len(mps_train)

    left_mpo_layers = [np.ones((1,1,1))]
    left_mps_layers = [np.ones((1,1))]

    # Вычисление тензоров слева направо
    for i in range(n-1):
        #print(i)
        #print(left_mps_layers[i].shape, b_mps[i].shape, b_mps[i+1].shape, right_mps_layers[-1-i].shape)
        local_A = compute_reduced_A(left_mpo_layers[i], right_mpo_layers[-1-i], mpo_train[i], mpo_train[i+1], optimize)
        local_b = compute_reduced_b(left_mps_layers[i], right_mps_layers[-1-i], b_mps[i], b_mps[i+1], optimize)
        solution = compute_pair_of_tensors(local_A, local_b, left_mps_layers[i].shape[-1], "left_to_right", eps, bond_limit)
        mps_train[i] = solution[0]
        if i < n-2:
            left_mpo_layers.append(compute_mpo_layer_from_left_to_right(left_mpo_layers[-1], [np.conj(mps_train[i]), mpo_train[i], mps_train[i]], optimize))
            left_mps_layers.append(compute_mps_layer_from_left_to_right(left_mps_layers[-1], [b_mps[i], np.conj(mps_train[i])], optimize))
    mps_train[-1] = solution[1]

    right_mpo_layers = [np.ones((1,1,1))]
    right_mps_layers = [np.ones((1,1))]

    # Вычисление тензоров справа налево
    for i in range(n-1):
        #print(left_mps_layers[-i-1].shape, b_mps[-i-2].shape, b_mps[-i-1].shape, right_mps_layers[i].shape)
        local_A = compute_reduced_A(left_mpo_layers[-i-1], right_mpo_layers[i], mpo_train[-i-2], mpo_train[-i-1], optimize)
        local_b = compute_reduced_b(left_mps_layers[-i-1], right_mps_layers[i], b_mps[-i-2], b_mps[-i-1], optimize)
        solution = compute_pair_of_tensors(local_A, local_b, right_mps_layers[i].shape[-1], "right_to_left", eps, bond_limit)
        mps_train[-i-1] = solution[1]
        if i < n-2:
            right_mpo_layers.append(compute_mpo_layer_from_right_to_left(right_mpo_layers[-1], [np.conj(mps_train[-i-1]), mpo_train[-i-1], mps_train[-i-1]], optimize))
            right_mps_layers.append(compute_mps_layer_from_right_to_left(right_mps_layers[-1], [b_mps[-i-1], np.conj(mps_train[-i-1])], optimize))
    mps_train[0] = solution[0]

    return mps_train, right_mps_layers, right_mpo_layers

def initialize_layers(mps_train, b_mps, mpo_train, optimize):
    
    n = len(mps_train)
    
    left_mpo_layers = [np.ones((1,1,1))]
    right_mpo_layers = [np.ones((1,1,1))]

    left_mps_layers = [np.ones((1,1))]
    right_mps_layers = [np.ones((1,1))]
    
    for i in range(1, n-1):
        right_mpo_layers.append(compute_mpo_layer_from_right_to_left(right_mpo_layers[-1], [np.conj(mps_train[-i]), mpo_train[-i], mps_train[-i]], optimize))    
        right_mps_layers.append(compute_mps_layer_from_right_to_left(right_mps_layers[-1], [b_mps[-i], np.conj(mps_train[-i])], optimize))
        
    return left_mpo_layers, right_mpo_layers, left_mps_layers, right_mps_layers

def initialize_mps_and_mpo(ex_mpo, ex_mps_right_part, ex_mps_initial_guess, eps, bond_limit):
    mps_train = mp.mps_compression(ex_mps_initial_guess.mps.copy(), eps, bond_limit)
    mpo_train = ex_mpo.mpo.copy()
    b_mps = mp.mps_compression(ex_mps_right_part.mps.copy())
    
    return mps_train, mpo_train, b_mps

In [4]:
def mp_solve5(ex_mpo, ex_mps_right_part, ex_mps_initial_guess, eps=1e-6, optimize=True, bond_limit=1000, sweep_number=1, renormalize=True):
    
    # Вычисление начальной нормы состояния
    initial_norm = mp.compute_norm(ex_mps_right_part.mps)
    # Инициализация MPO и MPS из объектов
    mps_train, mpo_train, b_mps = initialize_mps_and_mpo(ex_mpo, ex_mps_right_part, ex_mps_initial_guess, eps, bond_limit)
    # Первоначальная инициализация слоев сети
    left_mpo_layers, right_mpo_layers, left_mps_layers, right_mps_layers = initialize_layers(mps_train, b_mps, mpo_train, optimize)
    
    # Выполнение нескольких свипов вычисления решения
    for i in range(sweep_number):
        mps_train, right_mps_layers, right_mpo_layers = sweep(mps_train, b_mps, mpo_train, right_mps_layers, right_mpo_layers, eps, bond_limit, optimize)

    if renormalize:
        mp.renormalize_mps(mps_train, initial_norm)
        
    return MatrixProductState(mps_train)

In [5]:
chi = 1 / 2
theta = 0.001

t_general = 0.9
steps_number = 45
delta_t = t_general / steps_number

x_bond1 = 30
x_bond2 = 10

x_left = 10 + 2 * 10 * np.sqrt(2)
x_bias = 10 * np.sqrt(2)
x_right = 15

n1 = 12
n2 = 13

step1 = (x_right + x_left)/2**n1 #x_bond1 / 2**(n1 - 1)
step2 = x_bond2 / 2**(n2 - 1)

n3 = 10
x_bond3 = 10
step3 = x_bond3 / 2**(n3 - 1)

eps = 1e-6

# sp2 = np.linspace(-x_bond2, x_bond2, 2**n2)
# step2 = sp2[1]-sp2[0]

sp1 = np.linspace(x_bias-x_left, x_bias+x_right, 2**n1)
step1_v1 = sp1[1]-sp1[0]

initial_guess = ['approx', 1e-3]

bond_limit = 30

In [6]:
coh_operator_one = mp.der_mpo(n1, step1_v1)
coh_operator_two = mp.mpo_mul_num(mp.x_mpo(n1, x_bias - x_left, step1_v1), -1)
vac_operator_one = mp.mpo_sum(mp.mpo_multiplication(mp.x_mpo(n2, - x_bond2, step2), mp.x_mpo(n2, - x_bond2, step2)), mp.der2_mpo(n2, step2))
vac_operator_two = mp.mpo_sum(mp.one_mpo(n2), mp.mpo_mul_num(mp.mpo_multiplication(mp.x_mpo(n2, - x_bond2, step2), mp.der_mpo(n2, step2)), 2))
first_part = coh_operator_one + vac_operator_one
second_part = coh_operator_two + vac_operator_two
common_operator = mp.mpo_mul_num(mp.mpo_sum(first_part, second_part), chi / (np.sqrt(2)))
final_mpo = mp.mpo_sum(mp.one_mpo(n1 + n2), mp.mpo_mul_num(common_operator, delta_t))

In [7]:
initial_state_mps = mp.asym_gauss_mps(x_left, x_right, n1, 1) + mp.asym_gauss_mps(x_bond2, x_bond2, n2, 1)

In [ ]:
solution = mp.mp_solve(final_mpo, initial_state_mps, initial_state_mps, fullshape = True, eps = 1e-6, optimize = True, bond_limit = bond_limit, renormalize = True)

In [ ]:
ex_mpo = MatrixProductOperator(final_mpo)
ex_mps = MatrixProductState(initial_state_mps)
ex_guess = MatrixProductState(initial_state_mps)

In [ ]:
solution2 = mp_solve5(ex_mpo, ex_mps, ex_guess, eps=1e-6, optimize=True, bond_limit=bond_limit, sweep_number=1, renormalize=True)

In [ ]:
ar1 = mp.tensors_multiplication(initial_state_mps)
ar2 = mp.tensors_multiplication(solution2.mps)
ar3 = mp.tensors_multiplication(solution)

In [ ]:
array1 = ar1
array2 = ar2
cosine_similarity = np.dot(array1, array2) / (np.linalg.norm(array1) * np.linalg.norm(array2))
print(f"Косинусное сходство: {cosine_similarity}")

In [1]:
initial_guess

NameError: name 'initial_guess' is not defined

In [86]:
a_mpo = MatrixProductOperator(mp.der2_mpo(4))

In [87]:
mp.tensors_multiplication((a_mpo @ a).mps)

array([-2.46574857,  1.64504614,  1.68188347, -1.36456101,  0.40370209,
       -0.95220812,  0.10809326,  2.34168638, -2.60047836, -1.18930206,
        2.95694291, -0.98546849,  0.15995475, -0.79134872,  1.40903365,
       -1.05695943])

In [88]:
mp.tensors_multiplication((a @ a_mpo).mps)

array([-2.46574857,  1.64504614,  1.68188347, -1.36456101,  0.40370209,
       -0.95220812,  0.10809326,  2.34168638, -2.60047836, -1.18930206,
        2.95694291, -0.98546849,  0.15995475, -0.79134872,  1.40903365,
       -1.05695943])

In [92]:
(a_mpo @ a).bond

12

In [90]:
vector_1 @ mp.mpo_to_matrix(mp.der2_mpo(4))

array([-2.46574857,  1.64504614,  1.68188347, -1.36456101,  0.40370209,
       -0.95220812,  0.10809326,  2.34168638, -2.60047836, -1.18930206,
        2.95694291, -0.98546849,  0.15995475, -0.79134872,  1.40903365,
       -1.05695943])

In [410]:
print(c)

MPS(n_tensors=5, max_bond=2)


In [55]:
matrix_ex = np.random.randn(2,2) 
matrix_ex

array([[ 0.66726684,  0.7479502 ],
       [-1.20715724, -1.95351659]])

In [56]:
matrix_ex @ np.array([1, 2])

array([ 2.16316724, -5.11419042])

In [57]:
np.array([1, 2]) @ matrix_ex

array([-1.74704765, -3.15908298])

In [411]:
(vector_1-vector_2)

array([-1.08378316e+00,  9.93402094e-01,  1.31674387e-03, -2.57559127e-01,
       -1.08881689e-01,  2.75880745e+00, -1.09604885e+00, -6.03221365e-01,
        1.14253450e+00, -1.15453578e+00, -2.39658634e-01, -7.80868720e-01,
       -2.46340459e-01,  8.27612026e-01, -2.24878630e+00,  9.63069437e-01])

In [417]:
mp.tensors_multiplication((-(a-b)*2).mps)

array([ 2.16756632e+00, -1.98680419e+00, -2.63348774e-03,  5.15118255e-01,
        2.17763378e-01, -5.51761489e+00,  2.19209770e+00,  1.20644273e+00,
       -2.28506899e+00,  2.30907156e+00,  4.79317269e-01,  1.56173744e+00,
        4.92680918e-01, -1.65522405e+00,  4.49757260e+00, -1.92613887e+00])

In [5]:
vector_1 @ 5

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [414]:
a.bond, b.bond

(4, 4)

In [415]:
len(a)

4

In [416]:
print(a)

MPS(n_tensors=4.0, max_bond=4)


In [391]:
abs(a), np.linalg.norm(vector_1)

(np.float64(4.007321236660609), np.float64(4.007321236660607))

In [392]:
#vector = np.array([1, 2, 4, 5, 6, 3, 8, 5])
norm = np.linalg.norm(vector)
norm

np.float64(2.8879410167013564)

In [393]:
a.get_bond()

4

In [2]:
def mpo_x_mps(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        shape_0 = mpo[i].shape[0] * mps[i].shape[0]
        shape_2 = mpo[i].shape[-1] * mps[i].shape[-1]
        new_mps.append(np.einsum("ijnl, mno -> imjlo", mpo[i], mps[i], optimize=True).reshape(shape_0, 2, shape_2))
    return new_mps

In [53]:
def accelerated_mpo_x_mps(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        new_mps.append(oe.contract("ijnl, mno -> imjlo", mpo[i], mps[i]).reshape(mpo[i].shape[0] * mps[i].shape[0], 2, mpo[i].shape[-1] * mps[i].shape[-1]))
    return new_mps

In [3]:
#@jit(nopython=True)
def numba_mpo_x_mps(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        element = np.tensordot(mpo[i], mps[i], axes=(2,1))
        result = np.transpose(element, (0, 3, 1, 2, 4))
        new_mps.append(result.reshape(mpo[i].shape[0] * mps[i].shape[0], 2, mpo[i].shape[-1] * mps[i].shape[-1]))
    return new_mps

In [75]:
import numpy as np

def batch_mpo_x_mps(mpo, mps):
    """
    Вход:
    mpo - массив формы (batch_size, i, j, n, l)
    mps - массив формы (batch_size, m, n, o)
    
    Выход:
    массив формы (batch_size, i*m, j, l*o)
    """
    # Выполняем свертку по индексу n
    result = np.einsum('bijnl,bmno->bimjlo', mpo, mps, optimize='optimal')
    
    # Изменяем форму результата
    batch_size, i, m, j, l, o = result.shape
    return result.reshape(batch_size, i*m, j, l*o)

In [109]:
from numba import jit
import numpy as np

@jit
def numba_mpo_x_mps_dot(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        mp = mpo[i]
        ms = mps[i]
        i_dim, j_dim, n_dim, l_dim = mp.shape
        m_dim, n_dim_, o_dim = ms.shape
        
        # 1. Делаем массивы contiguous перед reshape
        mp_contig = np.ascontiguousarray(mp)
        ms_contig = np.ascontiguousarray(ms)
        
        # 2. Reshape с учетом порядка осей
        mp_2d = mp_contig.reshape((i_dim*j_dim*l_dim, n_dim))
        ms_2d = ms_contig.reshape((n_dim, m_dim*o_dim))
        
        # 3. Матричное умножение
        temp = np.dot(mp_2d, ms_2d)
        
        # 4. Reshape результата с преобразованием в contiguous
        result = np.ascontiguousarray(temp.reshape((i_dim, j_dim, l_dim, m_dim, o_dim)))
        
        # 5. Транспонирование и финальный reshape
        final = result.transpose((0, 3, 1, 2, 4)).reshape((i_dim*m_dim, j_dim, l_dim*o_dim))
        new_mps.append(final)
    
    return new_mps

In [114]:
from numba import njit
import numpy as np

@njit
def numba_mpo_x_mps_dot(mpo, mps):
    # Сначала проверим и преобразуем все массивы к единому формату
    batch_size = len(mps)
    
    # Определяем эталонные размерности по первому элементу
    i_dim, j_dim, n_dim, l_dim = mpo[0].shape
    m_dim = mps[0].shape[0]
    o_dim = mps[0].shape[2]
    
    # Создаем выходной массив заранее
    output = np.empty((batch_size, i_dim*m_dim, j_dim, l_dim*o_dim), dtype=np.float64)
    
    for b in range(batch_size):
        # Приводим массивы к contiguous и float64
        mp = np.ascontiguousarray(mpo[b].astype(np.float64))
        ms = np.ascontiguousarray(mps[b].astype(np.float64))
        
        # Основное вычисление
        for i in range(i_dim):
            for m in range(m_dim):
                for j in range(j_dim):
                    for l in range(l_dim):
                        for o in range(o_dim):
                            val = 0.0
                            for n in range(n_dim):
                                val += mp[i,j,n,l] * ms[m,n,o]
                            output[b, i*m_dim + m, j, l*o_dim + o] = val
    
    return output

In [77]:
def mpo_x_mps_optimized(mpo, mps):
    return [np.einsum("ijnl,mno->imjlo", mp, ms, optimize='optimal')
            .reshape(mp.shape[0]*ms.shape[0], 2, mp.shape[-1]*ms.shape[-1])
            for mp, ms in zip(mpo, mps)]

In [4]:
import numpy as np
from multiprocessing import Pool, cpu_count

def process_single_tensor(args):
    """Обработка одного тензора (для параллельного выполнения)."""
    mpo_i, mps_i = args
    result = np.einsum("ijnl, mno -> imjlo", mpo_i, mps_i)
    return result.reshape(mpo_i.shape[0] * mps_i.shape[0], 2, mpo_i.shape[-1] * mps_i.shape[-1])

def mpo_x_mps_parallel(mpo, mps, num_workers=None):
    """Параллельная версия mpo_x_mps."""
    if num_workers is None:
        num_workers = cpu_count()  # По умолчанию - все ядра
        
    with Pool(num_workers) as pool:
        new_mps = pool.map(process_single_tensor, zip(mpo, mps))
    return new_mps

In [104]:
j = 20
random_vector = np.random.randn(2**j)

In [105]:
#start = time.time()
%timeit mpo_x_mps(mp.der_mpo(j), mp.vec_to_mps(random_vector))
#print(time.time()-start)

3.31 s ± 47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
#start = time.time()
%timeit accelerated_mpo_x_mps(mp.der_mpo(j), mp.vec_to_mps(random_vector))
#print(time.time()-start)

3.17 s ± 61.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%timeit mpo_x_mps_optimized(mp.der_mpo(j), mp.vec_to_mps(random_vector))

3.37 s ± 90 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [116]:
%timeit numba_mpo_x_mps_dot(mp.der_mpo(j), mp.vec_to_mps(random_vector))

TypeError: can't unbox heterogeneous list: array(float64, 4d, C) != array(int64, 4d, C)

In [ ]:
import numpy as np
from multiprocessing import get_context

def process_single_tensor(args):
    mpo_i, mps_i = args
    result = np.einsum("ijnl, mno -> imjlo", mpo_i, mps_i)
    return result.reshape(mpo_i.shape[0] * mps_i.shape[0], 2, mpo_i.shape[-1] * mps_i.shape[-1])

def mpo_x_mps_parallel(mpo, mps, num_workers=2):
    ctx = get_context('spawn')
    with ctx.Pool(num_workers) as pool:
        return pool.map(process_single_tensor, zip(mpo, mps))

if __name__ == '__main__':
    random_vector = np.random.randn(2**18)
    start = time.time()
    mpo_x_mps_parallel(mp.der_mpo(18), mp.vec_to_mps(random_vector))
    print(time.time()-start)

In [ ]:
random_vector = np.random.randn(2**18)
start = time.time()
mpo_x_mps_parallel(mp.der_mpo(18), mp.vec_to_mps(random_vector))
print(time.time()-start)

In [46]:
import numpy as np

# Исходные данные (пример размеров)
A = np.random.rand(3, 3, 4, 5)  # Форма (i,j,n,l)
B = np.random.rand(2, 4, 6)     # Форма (m,n,o)

# 1. Свертка по оси n (индекс 2 в A и индекс 1 в B)
temp = np.tensordot(A, B, axes=(2, 1))  # Форма: (i,j,l,m,o)

# 2. Перегруппировка осей: (i,j,l,m,o) -> (i,m,j,l,o)
result = np.transpose(temp, (0, 3, 1, 2, 4))  # Нужный порядок: imjlo

In [47]:
einsum_result = np.einsum("ijnl,mno->imjlo", A, B)
tensordot_result = np.transpose(np.tensordot(A, B, axes=(2, 1)), (0, 3, 1, 2, 4))

np.allclose(einsum_result, tensordot_result)  # Должно вернуть True

True

In [ ]:
from multiprocessing import Pool
import numpy as np

def process_tensors(args):
    mp, ms = args
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

if __name__ == '__main__':  # Важно для Jupyter!
    mpo = [np.random.rand(2,2,3,3) for _ in range(10)]
    mps = [np.random.rand(2,3,3) for _ in range(10)]
    
    with Pool() as pool:
        results = pool.map(process_tensors, zip(mpo, mps))

In [ ]:
from multiprocessing import get_context
from multiprocessing import Pool
import numpy as np

def process_tensors(args):
    mp, ms = args
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

def parallel_processing(mpo, mps):
    ctx = get_context('spawn')  # Решает проблемы на Windows/Mac
    with ctx.Pool() as pool:
        return pool.map(process_tensors, zip(mpo, mps))

if __name__ == '__main__':
    mpo = [np.random.rand(2,2,3,3) for _ in range(10)]
    mps = [np.random.rand(2,3,3) for _ in range(10)]
    
    results = parallel_processing(mpo, mps)

In [15]:
import numpy as np
from pathos.multiprocessing import ProcessingPool

def process_tensors(args):
    import numpy as np
    mp, ms = args
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

mpo = [np.random.rand(2,2,3,3) for _ in range(10)]
mps = [np.random.rand(2,3,3) for _ in range(10)]

with ProcessingPool() as pool:
    results = pool.map(process_tensors, zip(mpo, mps))

ValueError: Pool not running

In [12]:
pool = ProcessingPool()
results = pool.map(process_tensors, zip(mpo, mps))
pool.close()

ValueError: Pool not running

In [2]:
from pathos.multiprocessing import ProcessingPool
import numpy as np

# Важно: функция должна содержать все необходимые импорты внутри
def process_tensors(args):
    import numpy as np  # Критический импорт внутри функции!
    mp, ms = args
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

# Создаем тестовые данные
mpo = [np.random.rand(2,2,3,3) for _ in range(10)]
mps = [np.random.rand(2,3,3) for _ in range(10)]

# Правильное создание и использование пула
try:
    # Вариант 1: Контекстный менеджер (рекомендуется)
    with ProcessingPool() as pool:
        results = pool.map(process_tensors, zip(mpo, mps))
    print(f"Успешно обработано {len(results)} тензоров")
    
except Exception as e:
    print(f"Ошибка: {str(e)}")
finally:
    # Гарантированное освобождение ресурсов
    if 'pool' in locals():
        pool.close()
        pool.join()

Ошибка: Pool not running


In [2]:
from pathos.multiprocessing import ProcessingPool
import numpy as np

def process_tensors(args):
    import numpy as np
    mp, ms = args
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

# Создаем и сразу закрываем пул в одном блоке
def run_parallel(mpo, mps):
    pool = None
    try:
        pool = ProcessingPool()
        results = pool.map(process_tensors, zip(mpo, mps))
        return results
    finally:
        if pool is not None:
            pool.close()
            pool.join()
            del pool  # Важно для Jupyter!

# Использование:
mpo = [np.random.rand(2,2,3,3) for _ in range(10)]
mps = [np.random.rand(2,3,3) for _ in range(10)]
results = run_parallel(mpo, mps)

In [1]:
import gc

def clean_parallel():
    for obj in gc.get_objects():
        if isinstance(obj, ProcessingPool):
            obj.close()
            obj.join()
            del obj

In [3]:
clean_parallel()
results = run_parallel(mpo, mps)

ValueError: Pool not running

In [4]:
# Первый запуск
results1 = run_parallel(mpo, mps)
print("Первый запуск:", len(results1))

# Второй запуск (должен работать)
results2 = run_parallel(mpo, mps)
print("Второй запуск:", len(results2))

ValueError: Pool not running

In [6]:
from joblib import Parallel, delayed
import numpy as np
import time

# Функция должна быть полностью самостоятельной (импорты внутри!)
def process_tensor_pair(mp, ms):
    import numpy as np  # Важно для параллельных процессов!
    return np.einsum('ijnl,mno->imjlo', mp, ms).reshape(mp.shape[0]*ms.shape[0], 2, -1)

# Данные
mpo = [np.random.rand(20,20,2,20) for _ in range(200)]  # 100 тензоров
mps = [np.random.rand(20,2,20) for _ in range(200)]

In [7]:
start_time = time.time()
results = Parallel(n_jobs=6)(delayed(process_tensor_pair)(mp, ms) for mp, ms in zip(mpo, mps))
print(time.time()-start_time)
#print(f"Обработано {len(results)} тензоров. Пример формы: {results[0].shape}")

MemoryError: 

In [ ]:
start_time = time.time()
result_without_boosting = [process_tensor_pair(mp, ms) for mp, ms in zip(mpo, mps)]
print(time.time()-start_time)

In [183]:
from joblib import Parallel, delayed
import numpy as np
from math import prod  # Для вычисления общего размера тензора

# Глобальные настройки
N_JOBS = 4  # Использовать 4 ядра
BATCH_SIZE = 10  # Число задач на один процесс

# Функция инициализации (выполняется в каждом процессе)
def init_worker():
    """Предзагружает модули в каждый worker"""
    import numpy as np
    import sys
    np.set_printoptions(threshold=5)  # Опционально: компактный вывод
    print(f"Worker {os.getpid()} ready") if DEBUG else None

# Основная функция обработки
def process_tensor_pair(mp, ms):
    """Вычисление одного тензора с предзагруженными модулями"""
    # Numpy уже загружен в init_worker
    result = np.einsum('ijnl,mno->imjlo', mp, ms, optimize='optimal')
    return result.reshape(mp.shape[0]*ms.shape[0], 2, -1)

# Параллельный запуск с предзагрузкой
def run_parallel(mpo, mps):
    with Parallel(n_jobs=N_JOBS, 
                 batch_size=BATCH_SIZE,
                 verbose=10,  # Логирование выполнения
                 prefer="processes",  # Для CPU-операций
                 max_nbytes=None,  # Без ограничения памяти
                 pre_dispatch='2*n_jobs') as parallel:
        
        # Инициализация каждого процесса
        results = parallel(
            delayed(process_tensor_pair)(mp, ms)
            for mp, ms in zip(mpo, mps)
        )
        return results

In [231]:
start_time = time.time()
results = run_parallel(mpo, mps)
print(time.time()-start_time)

0.0671694278717041


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


In [289]:
import numpy as np
np.__config__.show()  # Проверьте, используется ли OpenBLAS/MKL

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: C:/Users/Nik/miniconda3/envs/mp1/Library/include
    lib directory: C:/Users/Nik/miniconda3/envs/mp1/Library/lib
    name: blas
    openblas configuration: unknown
    pc file directory: D:\bld\numpy_1724748756526\_h_env\Library\lib\pkgconfig
    version: 3.9.0
  lapack:
    detection method: pkgconfig
    found: true
    include directory: C:/Users/Nik/miniconda3/envs/mp1/Library/include
    lib directory: C:/Users/Nik/miniconda3/envs/mp1/Library/lib
    name: lapack
    openblas configuration: unknown
    pc file directory: D:\bld\numpy_1724748756526\_h_env\Library\lib\pkgconfig
    version: 3.9.0
Compilers:
  c:
    commands: cl.exe
    linker: link
    name: msvc
    version: 19.29.30154
  c++:
    commands: cl.exe
    linker: link
    name: msvc
    version: 19.29.30154
  cython:
    commands: cython
    linker: cython
    name: cython
    version: 3.0.11
Machine Information:
  build

In [290]:
# Ограничьте потоки BLAS
import os
os.environ["OMP_NUM_THREADS"] = "1"  # Для OpenMP
os.environ["MKL_NUM_THREADS"] = "1"  # Для MKL

In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import MatrixProductFunctions as mp

# Исходные данные (NumPy-массивы)
a_np = np.random.randn(100, 100)
b_np = np.random.randn(100, 100)

# Конвертируем в JAX-массивы
a_jax = jnp.array(a_np)
b_jax = jnp.array(b_np)

# JIT-компилируем функцию с einsum
@jax.jit
def fast_einsum(a, b):
    return jnp.einsum('ij,jk->ik', a, b)

def usual_einsum(a, b):
    return np.einsum('ij,jk->ik', a, b)

# Первый вызов — компиляция (может быть медленным)
result = fast_einsum(a_jax, b_jax).block_until_ready()

# Дальше — быстро
%timeit fast_einsum(a_jax, b_jax).block_until_ready()

137 μs ± 4.62 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [2]:
%timeit usual_einsum(a_np, b_np)

464 μs ± 23.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [85]:
def mpo_x_mps(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        shape_0 = mpo[i].shape[0] * mps[i].shape[0]
        shape_2 = mpo[i].shape[-1] * mps[i].shape[-1]
        new_mps.append(np.einsum("ijnl, mno -> imjlo", mpo[i], mps[i], optimize=True).reshape(shape_0, 2, shape_2))
    return new_mps

In [86]:
@jax.jit
def jax_mpo_x_mps(mpo, mps):
    new_mps = []
    for i in range(len(mps)):
        shape_0 = mpo[i].shape[0] * mps[i].shape[0]
        shape_2 = mpo[i].shape[-1] * mps[i].shape[-1]
        new_mps.append(jnp.einsum("ijnl, mno -> imjlo", mpo[i], mps[i], optimize=True).reshape(shape_0, 2, shape_2))
    return new_mps

In [93]:
@jax.jit
def optimized_mpo_mps(mpo, mps):
    def process_site(mpo_t, mps_t):
        shape_0 = mpo_t.shape[0] * mps_t.shape[0]
        shape_2 = mpo_t.shape[-1] * mps_t.shape[-1]
        return jnp.einsum("ijnl,mno->imjlo", mpo_t, mps_t).reshape(shape_0, 2, shape_2)
    
    return jax.tree_map(process_site, mpo, mps)

In [96]:
j = 30
random_mps = [jax.numpy.array(np.random.randn(50,2,50)) for i in range(j)]

In [97]:
compressed_random_mps = random_mps

In [98]:
%timeit mpo_x_mps(mp.der_mpo(j), compressed_random_mps)

11.2 ms ± 216 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [99]:
#jax_mpo_x_mps(mp.der_mpo(j), compressed_random_mps)
%timeit jax_mpo_x_mps(mp.der_mpo(j), compressed_random_mps)

6.47 ms ± 277 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [100]:
%timeit result = optimized_mpo_mps(mp.der_mpo(j), compressed_random_mps)

6.34 ms ± 55.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
